# 機械学習コードスニペット

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

## カテゴリ変数の扱い

| 手法 | メリット | デメリット |
| :-: | :-: | :-: |
| ラベルエンコーディング: カテゴリを数値に変換する | カラムが増えない | 数値自体の大きさに意味はないので線形モデルやNN系モデルと相性は悪い |
| OneHotエンコーディング: カテゴリごとに1, 0に変換する | カテゴリの差分を厳密に区別して表現できる | カラム数がカテゴリの数だけ増える |

### カテゴリを数値に変換する
いわゆるラベルエンコーディングと呼ばれる方法.  
性別や出身地、デバイスの種類などに対して、1:1対応する数値に変換する.

`sklearn.preprocessing.OrdinalEncoder`を利用する.  

`{レコード数} * {対象のカラム数}`の形の配列系のデータを投入できる.

In [2]:
# サンプルデータ
df = pd.DataFrame([["a"], ["b"], ["c"]], columns=["col"])
print("元データ")
display(df)

# 変換処理
oe = OrdinalEncoder()
print("変換したデータ")
display(
    df.assign(
        encoded=oe.fit_transform(df[["col"]]),
    )
)

元データ


,col
0,a
1,b
2,c


変換したデータ


,col,encoded
0,a,0.0
1,b,1.0
2,c,2.0


### 不明な値がある場合

`handle_unknown`で処理を変える
- `error`: デフォルト値. エラーになる
- `use_encoded_value`: `unknown_value`で指定.



In [3]:
try:
    df = pd.DataFrame([["a"], ["b"], ["d"]], columns=["col"])

    display(
        df.assign(
            encoded=oe.transform(df[["col"]]),
        )
    )
except:
    print("エラー")


# サンプルデータ
df = pd.DataFrame([["a", "a"], ["b", "b"], ["c", "d"]], columns=["col1", "col2"])
print("元データ")
display(df)

# 変換処理 handle_unknownを指定
oe = OrdinalEncoder(handle_unknown="use_encoded_value", dtype=int, unknown_value=-1)

# サンプルのためにカラム名の異なるデータで実行するので、ndarrayに変換しておく
oe.fit(df[["col1"]].values)
print("変換したデータ")
display(
    df.assign(
        encoded_col2=oe.transform(df[["col2"]].values),
    )
)

エラー
元データ


,col1,col2
0,a,a
1,b,b
2,c,d


変換したデータ


,col1,col2,encoded_col2
0,a,a,0
1,b,b,1
2,c,d,-1


- dに対して、-1が適応されている
- dtypeを指定することで整数型に明示的に変更できる

### カテゴリごとに1, 0に変換する
いわゆるOneHotEncodingと呼ばれる手法.  
カテゴリの種類ごとに対象のカテゴリであれば1、そうでなければ0であるカラムを作成する.

`sklearn.preprocessing.OneHotEncoder`を利用して実現できる

変換後の値はカテゴリの数だけ列が増えて返却されるので、扱い方に少し気をつける必要がある.


メモ
- `sparse_output`はデフォルトで`True`だけど`False`にした方が扱いやすいかも
- 出力のカラム名はEncoder側がいい感じにしてくれる. `oe.get_feature_names_out()`で取得できる.
- 未知の値の扱いについては、`OrdinalEncoder`と同じ

In [4]:
# サンプルデータ
df = pd.DataFrame([["a"], ["b"], ["c"]], columns=["col"])
print("元データ")
display(df)

# 変換処理
oe = OneHotEncoder(sparse_output=False)
print("変換したデータ")
encoded = oe.fit_transform(df[["col"]])
display(
    pd.concat([df, pd.DataFrame(encoded, columns=oe.get_feature_names_out())], axis=1)
)

元データ


,col
0,a
1,b
2,c


変換したデータ


,col,col_a,col_b,col_c
0,a,1.0,0.0,0.0
1,b,0.0,1.0,0.0
2,c,0.0,0.0,1.0


## 順序ありカテゴリ
成績(S, A, B, C)や格付けランクなど直接数値では表現されていないが、順序があるもの  

基本は数値に変換する対応でいい.

- `pandas`: `DataFrame.replace`
- `polars`: `Expr.replace`

polarsは0.20の以降の関数で0.20より小さいバージョンでは`map_dict`

変換の指定方法はいろいろあるけど、一番直感的だと思うやつをとりあえず記載する.


In [5]:
df = pd.DataFrame([["a"], ["b"], ["c"]], columns=["col"])
print("変換元のデータ")
display(df)

print("置換後のデータ")
# a -> b -> cの順に1, 2, 3に置換する
replace_map = {"a": "1", "b": "2", "c": "3"}
display(
    pd.concat(
        [
            df,
            df.replace({"col": replace_map}),
        ],
        axis=1,
    )
)

変換元のデータ


,col
0,a
1,b
2,c


置換後のデータ


,col,col
0,a,1
1,b,2
2,c,3


## 数値カラム

## クリッピング
極端な外れ値を消すためにカラムに上限と下限をつける

In [10]:
df = pd.DataFrame({"col": np.arange(-100, 100, 25)})
print("元データ")
display(df)

print("-20 ~ 20に変換する")
display(
    df.clip(-20, 20)
)

元データ


,col
0,-100
1,-75
2,-50
3,-25
4,0
5,25
6,50
7,75


-20 ~ 20に変換する


,col
0,-20
1,-20
2,-20
3,-20
4,0
5,20
6,20
7,20


### 量子化
連続値を持つカラムを一定の値にまとめる.  
イメージとしては四捨五入のようなイメージ.

- 等分割の場合はシンプルで、整数割をした上で、割った数をかければいい. `x // n * n`
- 任意の値で分割したい場合は少し処理がややこしい


In [42]:
df = pd.DataFrame({"変換前": np.arange(-10, 10)})

print("3ずつに変換")
df["変換後"] = df["変換前"] // 3 * 3

display(df)

3ずつに変換


,変換前,変換後
0,-10,-12
1,-9,-9
2,-8,-9
3,-7,-9
4,-6,-6
5,-5,-6
6,-4,-6
7,-3,-3
8,-2,-3
9,-1,-3


自由に閾値を決める例

もっといいやり方はあるかもしれない.

In [45]:
# 0と5を区切りの点とする
thresholds = [0, 5]

temp = pd.DataFrame(np.zeros_like(df["変換前"]), columns=["temp"])
for th in thresholds:
    temp["temp"] += (df["変換前"] >= th).astype(int)

temp.replace({
    "temp": {0: df["変換前"].min()} | {idx: value for idx, value in enumerate(thresholds, 1)}
}) 

,temp
0,-10
1,-10
2,-10
3,-10
4,-10
5,-10
6,-10
7,-10
8,-10
9,-10
